In [1]:
from bayes_opt import BayesianOptimization
import numpy as np
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from matplotlib import gridspec

from trainlib.ConfigFileHandler import ConfigFileHandler

In [2]:
def load_file(path, keys):
    confhandler = ConfigFileHandler()
    confhandler.load_configuration(path)
    
    retval = {}
    
    for section_name in confhandler.get_sections():
        cur_section = confhandler.get_section(section_name)
        
        for key in keys:
            if not key in retval:
                retval[key] = []
                
            retval[key].append(float(cur_section[key]))
            
    return retval

In [3]:
def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    return mu, sigma

def plot_gp(bo, x):
    
    fig = plt.figure(figsize=(16, 10))
    axis = plt.subplot(111)
    
    mu, sigma = posterior(bo, x)
    axis.plot(bo.X.flatten(), bo.Y, 's', markersize=8, label=u'observations', color='black')
    axis.plot(x, mu, '--', color='black', label='gaussian process mean')

    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
        alpha=.6, fc='lightblue', ec='None', label='95% confidence interval')
    
    axis.set_xlim((min(x), max(x)))
    axis.set_ylim((None, None))
    axis.set_ylabel('utility', fontdict={'size':20})
    axis.set_xlabel('ggH prior', fontdict={'size':20})
    
    axis.legend(loc = 3)
    
    return fig

In [4]:
init_dict = load_file("/data_CMS/cms/wind/180427_combined_optimized_inputs/number_layers_2.0_number_neurons_64.0/priors_tree/evaluations_ggH.txt", ["target", "ggh_prior"])

attempting to load configuration file from /data_CMS/cms/wind/180427_combined_optimized_inputs/number_layers_2.0_number_neurons_64.0/priors_tree/evaluations_ggH.txt


In [6]:
gp_params = {'kernel': 1.0 * Matern(length_scale = 0.01, length_scale_bounds = (1e-5, 1e5), nu = 1.5),
             'alpha':1.5e-6}

In [7]:
target = lambda x: x

In [11]:
x = np.linspace(1.0, 3.0, 1000).reshape(-1,1)

In [12]:
bo = BayesianOptimization(target, {'ggh_prior': (1.0, 3.0)})

In [13]:
bo.initialize(init_dict)

In [14]:
bo.maximize(init_points=0, n_iter=0, acq='poi', kappa=5, xi = 0.1, **gp_params)

Initialization
-------------------------------------------
 Step |   Time |      Value |   ggh_prior | 
    1 | 00m00s |   -0.00440 |      2.8029 | 
    2 | 00m00s |    0.01713 |      1.4256 | 
    3 | 00m00s |    0.01757 |      1.4287 | 
    4 | 00m00s |    0.01898 |      2.0435 | 
    5 | 00m00s |    0.01597 |      1.0000 | 
    6 | 00m00s |    0.01774 |      1.3472 | 
    7 | 00m00s |    0.00534 |      2.6094 | 
    8 | 00m00s |    0.01724 |      1.5516 | 
    9 | 00m00s |    0.01726 |      1.1984 | 
   10 | 00m00s |    0.01812 |      1.2680 | 
   11 | 00m00s |    0.01792 |      1.7238 | 
   12 | 00m00s |    0.00612 |      2.6118 | 
   13 | 00m00s |    0.01519 |      2.1811 | 
   14 | 00m00s |   -0.01159 |      2.9733 | 
   15 | 00m00s |    0.01479 |      2.1885 | 
   16 | 00m00s |    0.01822 |      1.5799 | 
   17 | 00m00s |    0.01766 |      1.7530 | 
   18 | 00m00s |    0.01817 |      2.0919 | 
   19 | 00m00s |    0.01469 |      2.2500 | 
   20 | 00m00s |    0.00448 |      2.6014

In [15]:
fig = plot_gp(bo, x)

In [16]:
plt.savefig("gaussian_process.pdf")